## Credentials and also connectivity parameters

In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import random

# Define connection parameters for both databases
ssms_connection_params = {
    'drivername': "mssql+pyodbc",
    'username': "sphinxsolix",
    'password': "uwSfGY{]d'a1NB+[v#cD",
    'host': "AMS-P-SQL04\SQL04",
    'port': "4204",
    'database': "AE_Live",
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

mysql_connection_params = {
    'drivername': 'mysql+pymysql',
    'username': 'NAVEURO',
    'password': 'NAVEURO_2024!',
    'host': 'tow-p-solix01',
    'port': '3307',
    'database': 'SPHINX_NAV_EURO',  
}

# Create database engine instances
ssms_engine = create_engine(f"{ssms_connection_params['drivername']}://"
                            f"{ssms_connection_params['username']}:{ssms_connection_params['password']}@"
                            f"{ssms_connection_params['host']}:{ssms_connection_params['port']}/"
                            f"{ssms_connection_params['database']}?driver={ssms_connection_params['query']['driver']}")

mysql_engine = create_engine(f"{mysql_connection_params['drivername']}://"
                             f"{mysql_connection_params['username']}:{mysql_connection_params['password']}@"
                             f"{mysql_connection_params['host']}:{mysql_connection_params['port']}/"
                             f"{mysql_connection_params['database']}")


## V1 - Validation for the total number of Rows count in source and Target..!

In [3]:
# Specify the table names from source and target
source_table_name = 'Object Metadata'
target_table_name = f"{source_table_name}_AE_LIVE"

# Count rows in source table
source_row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM [{source_table_name}];", ssms_engine).iloc[0, 0]

# Count rows in target table
target_row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM `{target_table_name}`;", mysql_engine).iloc[0, 0]

# Validate row count
if source_row_count == target_row_count:
    print(f"Row count matches, Source: {source_row_count}, Target: {target_row_count}, Table Name - {target_table_name}")
else:
    print("Row count doesn't match. Check migration process.")
    
# Dispose the engine instances
ssms_engine.dispose()
mysql_engine.dispose()


Row count matches, Source: 7796, Target: 7796, Table Name - Object Metadata_AE_LIVE


## V2  Validation for the total number of Columns count in source and Target..!

In [5]:
# Specify the table names from source and target
source_table_name = 'Object Metadata'
target_table_name = f"{source_table_name}_AE_LIVE"

# Get column names from source table
source_columns = pd.read_sql_query(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{source_table_name}';", ssms_engine)['COLUMN_NAME'].tolist()

# Get column names from target table
target_columns = pd.read_sql_query(f"SHOW COLUMNS FROM `{target_table_name}`;", mysql_engine)['Field'].tolist()

# Validate column count
if len(source_columns) == len(target_columns):
    print(f"Column count matches. Source: {len(source_columns)}, Target: {len(target_columns)}, Table Name - {target_table_name}")
else:
    print("Column count doesn't match. Check migration process.")

# Dispose the engine instances
ssms_engine.dispose()
mysql_engine.dispose()


Column count matches. Source: 8, Target: 8, Table Name - Object Metadata_AE_LIVE


## V3 Validation for the Sum of numeric Columns..!!

In [6]:
# Specify the table names from source and target
source_table_name = 'Object Metadata'
target_table_name = f"{source_table_name}_AE_LIVE"

# Get numeric column names from source table
source_numeric_columns = pd.read_sql_query(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{source_table_name}' AND DATA_TYPE IN ('INT', 'FLOAT', 'DECIMAL');", ssms_engine)['COLUMN_NAME'].tolist()

# Get numeric column names from target table
target_numeric_columns = pd.read_sql_query(f"SHOW COLUMNS FROM `{target_table_name}` WHERE Type IN ('INT', 'FLOAT', 'DECIMAL');", mysql_engine)['Field'].tolist()

# Validate column count
if len(source_numeric_columns) != len(target_numeric_columns):
    print("Numeric column count doesn't match. Check migration process.")
else:
    # Calculate sums for numeric columns from source table
    source_sums = {}
    for col in source_numeric_columns:
        query = f"SELECT SUM(`{col}`) FROM `{source_table_name}`;"
        source_sums[col] = pd.read_sql_query(query, ssms_engine).iloc[0, 0]

    # Calculate sums for numeric columns from target table
    target_sums = {}
    for col in target_numeric_columns:
        query = f"SELECT SUM(`{col}`) FROM `{target_table_name}`;"
        target_sums[col] = pd.read_sql_query(query, mysql_engine).iloc[0, 0]

    # Validate sums
    sums_matched = all(source_sums[col] == target_sums[col] for col in source_sums)
    if sums_matched:
        print("Sums for numeric columns match.")
    else:
        print("Sums for numeric columns don't match. Check migration process.")

# Dispose the engine instances
ssms_engine.dispose()
mysql_engine.dispose()










# This code iterates through each numeric column in the source table, calculates the sum for each column in both 
# the source and target tables, and compares them individually. If any of the sums do not match, it will print which columns 
# have mismatched sums. If all sums match, it will print a success message indicating that the validation was successful. 
# This approach provides a more thorough verification of the numeric column sums between the source and target tables.


Sums for numeric columns match.


## V4 Validation for the sum of length of characters  in columns

In [ ]:
# Specify the table names from source and target
source_table_name = 'Object Metadata'
target_table_name = f"{source_table_name}_AE_LIVE"


# Check for differences in column data
for column in source_columns:
    ssms_query = f"SELECT TOP 3 [{column}] FROM [{source_table_name}];"  # Adjust for SQL Server
    mysql_query = f"SELECT `{column}` FROM `{target_table_name}` LIMIT 3;"
    ssms_data = pd.read_sql_query(ssms_query, ssms_engine)
    mysql_data = pd.read_sql_query(mysql_query, mysql_engine)
    
    # Calculate sum of characters in each column's data
    ssms_sum = ssms_data.apply(lambda x: x.str.len()).sum()
    mysql_sum = mysql_data.apply(lambda x: x.str.len()).sum()
    
    # Compare the sums
    if ssms_sum.equals(mysql_sum):
        print(f"Sum of characters in column '{column}' matches.")
    else:
        print(f"Sum of characters in column '{column}' doesn't match.")
        
        # Print a sample of data for columns with mismatched sums
        print("Sample data (SSMS):")
        print(ssms_data.sample(min(3, len(ssms_data))))  # Print a maximum of 3 rows or all rows if there are fewer
        print("Sample data (MySQL):")
        print(mysql_data.sample(min(3, len(mysql_data))))  # Print a maximum of 3 rows or all rows if there are fewer


## V5 Validations for the sum of "LONG RAW, LONG, CLOB, BLOB" columns

## Validations in a single code

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError

# Define connection parameters for both databases
ssms_connection_params = {
    'drivername': "mssql+pyodbc",
    'username': "sphinxsolix",
    'password': "uwSfGY{]d'a1NB+[v#cD",
    'host': "AMS-P-SQL04\SQL04",
    'port': "4204",
    'database': "AE_Live",
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

mysql_connection_params = {
    'drivername': 'mysql+pymysql',
    'username': 'NAVEURO',
    'password': 'NAVEURO_2024!',
    'host': 'tow-p-solix01',
    'port': '3307',
    'database': 'SPHINX_NAV_EURO',  
}

# Create database engine instances
ssms_engine = create_engine(f"{ssms_connection_params['drivername']}://"
                            f"{ssms_connection_params['username']}:{ssms_connection_params['password']}@"
                            f"{ssms_connection_params['host']}:{ssms_connection_params['port']}/"
                            f"{ssms_connection_params['database']}?driver={ssms_connection_params['query']['driver']}")

mysql_engine = create_engine(f"{mysql_connection_params['drivername']}://"
                             f"{mysql_connection_params['username']}:{mysql_connection_params['password']}@"
                             f"{mysql_connection_params['host']}:{mysql_connection_params['port']}/"
                             f"{mysql_connection_params['database']}")

# Read table names from the CSV file
tables_df = pd.read_csv("C:\\Users\\HXK0531A\\Desktop\\test_tables.csv")

# Initialize an empty list to store validation results for all tables
all_validation_results = []

# Iterate over each table and perform validations
for index, column in tables_df.iterrows():
    source_table_name = column['table_name']
    target_table_name = f"{source_table_name}_AE_LIVE"
    
    try:
        # Check if the target table exists
        pd.read_sql_query(f"SELECT 1 FROM `{target_table_name}` LIMIT 1;", mysql_engine)
    except ProgrammingError:
        # If target table doesn't exist, mark all validations as failed
        validation_results = {
            'V1': 'Fail',
            'V2': 'Fail',
            'V3': 'Fail',
            'V4': 'Fail',
            'Table Name': target_table_name
        }
        all_validation_results.append(validation_results)
    else:
        # Get column names from source table
        source_columns = pd.read_sql_query(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{source_table_name}';", ssms_engine)['COLUMN_NAME'].tolist()

        # Get column names from target table
        target_columns = pd.read_sql_query(f"SHOW COLUMNS FROM `{target_table_name}`;", mysql_engine)['Field'].tolist()

        # Get numeric column names from source table
        source_numeric_columns = pd.read_sql_query(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{source_table_name}' AND DATA_TYPE IN ('INT', 'FLOAT', 'DECIMAL');", ssms_engine)['COLUMN_NAME'].tolist()

        # Get numeric column names from target table
        target_numeric_columns = pd.read_sql_query(f"SHOW COLUMNS FROM `{target_table_name}` WHERE Type IN ('INT', 'FLOAT', 'DECIMAL');", mysql_engine)['Field'].tolist()

        # Check for differences in column data
        column_validation_results = {}
        for column in source_columns:
            ssms_query = f"SELECT TOP 3 [{column}] FROM [{source_table_name}];"  # Adjust for SQL Server
            mysql_query = f"SELECT `{column}` FROM `{target_table_name}` LIMIT 3;"
            ssms_data = pd.read_sql_query(ssms_query, ssms_engine)
            mysql_data = pd.read_sql_query(mysql_query, mysql_engine)

            # Check if the column contains string data
            if ssms_data.dtypes[0] == 'object':
                # Calculate sum of characters in each column's data
                ssms_sum = ssms_data.apply(lambda x: x.str.len()).sum()
                mysql_sum = mysql_data.apply(lambda x: x.str.len()).sum()

                # Compare the sums
                if ssms_sum.equals(mysql_sum):
                    column_validation_results[column] = 'Pass'
                else:
                    column_validation_results[column] = 'Fail'
            else:
                # If not string data, compare directly
                if ssms_data.equals(mysql_data):
                    column_validation_results[column] = 'Pass'
                else:
                    column_validation_results[column] = 'Fail'

        # Validate row count
        source_row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM [{source_table_name}];", ssms_engine).iloc[0, 0]
        target_row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM `{target_table_name}`;", mysql_engine).iloc[0, 0]
        row_count_validation = 'Pass' if source_row_count == target_row_count else 'Fail'

        # Validate column count
        column_count_validation = 'Pass' if len(source_columns) == len(target_columns) else 'Fail'

        # Validate numeric column count
        numeric_column_count_validation = 'Pass' if len(source_numeric_columns) == len(target_numeric_columns) else 'Fail'

        # Validate sums for numeric columns
        sums_matched = all(pd.read_sql_query(f"SELECT SUM(`{col}`) FROM `{source_table_name}`;", ssms_engine).iloc[0, 0] ==
                           pd.read_sql_query(f"SELECT SUM(`{col}`) FROM `{target_table_name}`;", mysql_engine).iloc[0, 0]
                           for col in source_numeric_columns)
        numeric_sum_validation = 'Pass' if sums_matched else 'Fail'

        # Add validation results to the list
        validation_results = {
            'V1': row_count_validation,
            'V2': column_count_validation,
            'V3': numeric_column_count_validation,
            'V4': numeric_sum_validation,
            'Table Name': target_table_name
        }
        all_validation_results.append(validation_results)

# Add missing tables to validation results with all failed validations
missing_tables = set(tables_df['table_name']) - set([result['Table Name'].replace('_AE_LIVE', '') for result in all_validation_results])
for table in missing_tables:
    validation_results = {
        'V1': 'Fail',
        'V2': 'Fail',
        'V3': 'Fail',
        'V4': 'Fail',
        'Table Name': f"{table}_AE_LIVE"
    }
    all_validation_results.append(validation_results)

# Combine all validation results into a single DataFrame
combined_df = pd.DataFrame(all_validation_results)

# Write the combined validation results to a single CSV file
combined_df.to_csv('validation_results.csv')
